# Extended RNA-Seq Analysis Training Demo

## Overview

In this tutorial, we will repeat the Tutorial 1B extended submodule with a different dataset. Using the provided dataset is a great way to get introduced to the module and learn its basic functionality, but to use it with your own data or to understand the module at a deeper level, it is helpful to practice adapting it to a new dataset. We will guide you through that here, providing basic guidance on the workflow and letting you write the commands yourself. If you get stuck, you can always view our hints and suggestions by clicking on the dropdown arrow under the command cells.

The initial dataset used a prokaryotic sample for simplicity and compute efficiency. This time, we will increase the complexity a bit by using a eukaryotic sample. We have selected data from a time series experiment on Plasmodium falciparum, the parasite responsible for malaria. The experiment did RNA-seq analysis on P. falciparum cells at 25 time points after erythrocyte invasion. The data is taken from Kucharski M, Tripathi J, Nayak S, Zhu L et al. A comprehensive RNA handling and transcriptomics guide for high-throughput processing of Plasmodium blood-stage samples. Malar J 2020 Oct 9;19(1):363. The sequence data is available from SRA with the accession number [SRP261441](https://www.ncbi.nlm.nih.gov/sra/?term=SRP261441). To keep things simple, we are not using all time points, and have selected time points 1, 13, and 25 as a proxy for early, mid, and late infection. Feel free to add or remove samples from your analysis to see how the results differ. The workflow structure remains the same as the original submodule 1B, with the diagram shown below.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Install Mambaforge and then install snakemake using bioconda.

First install Mambaforge.


In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
    
  Make sure you include the `!` in front of your command!  
    

`curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh` 
`bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge` 
`date +"%T"` 

</details>




Next, add it to the path

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
    
    

`#add to your path`  
`import os`  
`os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"`  


</details>




Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
    

`mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc salmon entrez-direct gffread parallel-fastq-dump sra-tools pigz`


</details>

### STEP 2: Setup Environment

The directory structure will be the same as in Tutorial 1B. Create that below.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
    

` cd $HOMEDIR`  
` mkdir -p data`  
` mkdir -p data/raw_fastq`  
` mkdir -p data/trimmed`  
` mkdir -p data/fastqc`  
` mkdir -p data/aligned`  
` mkdir -p data/reference`  

</details>

Set number of cores depending on your VM size

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
  

`numthreads=!nproc`  
`numthreadsint = int(numthreads[0])`  
`%env CORES = $numthreadsint`  
`echo ${CORES}`  
    
</details>



### STEP 3: Downloading relevant FASTQ files using SRA Tools

As before, we need to download FASTQ data from SRA. We mentioned at the beginning of this tutorial that we are only using 3 out of the 25 time points from the experiment. The Sequence Run Project for this manuscript is [SRP261441](https://www.ncbi.nlm.nih.gov/sra/?term=SRP261441). We're going to select time points 1, 13, and 25. Look through the SRP page and find the accession numbers for those time points. We will use them in the next step.

<details>
  <summary>Click for help</summary>
    
    Accession numbers are SRR11784387, SRR11784398, and SRR11784410
  
</details>


Add those accession numbers to your accs.txt file and have a look at it to make sure they are correct.

In [ ]:
cat accs.txt

### Downloading multiple files using the SRA-toolkit.

Next we are going to download the FASTQ files using the sra-toolkit. We are going to do this in two steps: prefetch followed by fasterq-dump. Write the command below that will prefetch the data.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   
    
   `prefetch -O data/raw_fastq/ --option-file accs.txt `
   
  
</details>

### Converting Multiple SRA files to Fastq

As before, use a loop to convert each file in the list.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`for x in `cat accs.txt`; do fasterq-dump -f -O data/raw_fastq -e $CORES -m 4G data/raw_fastq/$x/$x.sra; done`  

   
  
</details>

Convert to fastq.gz

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`pigz data/raw_fastq/*.fastq`  

   
  
</details>

### STEP 4: Copy reference transcriptome files that will be used by Salmon using E-Direct

As before, we are going to use Salmon to quantify the transcriptome. We will need a Refseq assembly here. Since these reads are on the _Plasmodium falciparum_ organism, we need to update our Refseq files. Try searching through [NCBI genome database](https://www.ncbi.nlm.nih.gov/datasets/genome) for the appropriate _Plasmodium falciparum_ assembly.

Once you find an assembly, download transcriptome reference files from that assembly using FTP links.

Try repurposing the esearch commands from Tutorial1B to download the files for _P. falciparum_. 

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`#parse for the ftp link and download the genome reference fasta file`  
`esearch -db assembly -query GCF_000002765.6 | efetch -format docsum \`  
`| xtract -pattern DocumentSummary -element FtpPath_RefSeq \`  
`| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.fna.gz " $0"/"$NF"_genomic.fna.gz"}' \`  
`| bash`  

`#parse for the ftp link and download the gtf reference fasta file`  
`esearch -db assembly -query GCF_000002765.6 | efetch -format docsum \`  
`| xtract -pattern DocumentSummary -element FtpPath_RefSeq \`  
`| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.gff.gz " $0"/"$NF"_genomic.gff.gz"}' \`  
`| bash`  

`# parse for the ftp link and download the feature-table reference file`  
`# (for later use for merging readcounts with gene names in R code).`
`esearch -db assembly -query GCF_000002765.6 | efetch -format docsum \`  
`| xtract -pattern DocumentSummary -element FtpPath_RefSeq \`  
`| awk -F"/" '{print "curl -o data/reference/"$NF"_feature_table.txt.gz " $0"/"$NF"_feature_table.txt.gz"}' \`  
`| bash`  


`#unzip the compresseed fasta files`  
`gzip -d data/reference/*.gz --force`  
   
  
</details>










Now create a transcriptome reference file from the files we downloaded using the `gffread` command.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`gffread -w data/reference/GCF_000002765.6_transcriptome_reference.fa -g data/reference/GCF_000002765.6_GCA_000002765_genomic.fna`   `data/reference/GCF_000002765.6_GCA_000002765_genomic.gff`  

   
  
</details>


It is also recommended to include the full genome at the end of the transcriptome reference file, for the purpose of performing a 'decoy-aware' mapping, more information about which can be found in the Salmon documentation. Note that a key difference between this and the first tutorial is that since we are using data from a eukaryotic organism, the genome comes in multiple chromosomes. That means that we will actually be pasting 14 sequences (_P. falciparum_ has 14 chromosomes) to the transcriptome file and writing 14 lines to the decoy file.

There are a couple ways you could do this. One easy whay would be to simply copy and past the chromosome names into the decoys.txt file, but this does scale well if you end up using an organism with many chromosomes or if you make a user error. A better way is to programmatically extract the chromosome names from the genome fasta file. Try combining some `grep` and `sed` commands to pull the ID lines from the fasta file and extract the chromosome name from it. If you get stuck, we have one option in the hint box below.

<details>
  <summary>Click for help</summary>
   

`# First use cat to combine the transcriptome and genome fasta files into a transcriptome_reference_w_decoy fasta file`  
`cat data/reference/GCF_000002765.6_transcriptome_reference.fa <(echo) data/reference/GCF_000002765.6_GCA_000002765_genomic.fna > data/reference/GCF_000002765.6_transcriptome_reference_w_decoy.fa`  

`# Next, pull out the fasta headers by searching for > and extract the chromosome name from the line of text.`  
`cat data/reference/GCF_000002765.6_GCA_000002765_genomic.fna | grep ">" | sed 's/Plasmodium.*//g' | sed 's/>//g' > data/reference/decoys.txt`  
   
  
</details>

### STEP 5: Copy data file for Trimmomatic

Using the `gsutil` command, copy the TruSeq3-PE.fa file to your working directory. For convenience, a copy of this file is kept at `gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/`

In [ ]:
<YOUR COMMAND HERE>

### STEP 6: Run Trimmomatic
Next write a command to run Trimmomatic on each of the sequences. 

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

You can create a sort of loop by using the `cat` command to read the accession numbers from the `accs.txt` file and piping it into Trimmomatic. Use the `xargs` command to get the output from `cat` into Trimmomatic.
   
  
</details>



### STEP 7: Run FastQC
Next we run FastQC. You can use the same `cat` command as above to run each sample through FastQC successively.

In [ ]:
<YOUR COMMAND HERE>

### STEP 8: Run MultiQC
Create a MultiQC report from your output.

In [ ]:
<YOUR COMMAND HERE>

### STEP 9: Index the Transcriptome so that Trimmed Reads Can Be Mapped Using Salmon

Over the next few steps we need to index the transcriptome, map the reads to the transcriptom, and quantify the mapping. Using the `salmon index` command, create a Salmon index using the files we assembled above. Try using `salmon index -h` if you need help.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`salmon index -t data/reference/GCF_000002765.6_transcriptome_reference_w_decoy.fa -p $CORES -i data/reference/transcriptome_index --decoys data/reference/decoys.txt -k 31 --keepDuplicates`  
   
  
</details>


### STEP 10: Run Salmon to Map Reads to Transcripts and Quantify Expression Levels
Run Salmon on each sample using `salmon quant`. You can either create a `for` loop, or use `cat accs.txt` to pipe each sample into Salmon.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`cat accs.txt | xargs -I {} salmon quant -i data/reference/transcriptome_index -l SR -r "data/trimmed/{}_1_trimmed.fastq.gz" -p $CORES --validateMappings -o "data/quants/{}_quant"`  

   
  
</details>


### STEP 11: Report the top 10 most highly expressed genes in the samples

In Tutorial 1B, several examples are shown of using `head`, `sort`, and `grep` to explore the genes that are highly up- or down-regulated in the samples. Try doing some of that exploring here. Additionally, you can have a look at the paper that produced the data to see what kind of genes they explored and see if you can replicate their results.

In [ ]:
<YOUR COMMAND HERE>

### STEP 12: Combine Genecounts to a Single Genecount File
As before, create a readcounts table that combines the counts for each gene across each sample into one easy to analyze quants file. Salmon provides the `salmon quantmerge` command to do this.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
   

`# First merge salmon files by number of reads.`  
`salmon quantmerge --column numreads --quants data/quants/*_quant -o data/quants/merged_quants.txt`  

`# Rename the columns based on yoru samples`  


`#Print the new dataframe`  
`print("An example of a combined genecount outputfile.")`  
`!head data/quants/merged_quants.txt`  
   
  
</details>


## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the other notebooks using this dataset. What changes will you need to make in order to make those notebooks work with the new data?